In [1]:
#from part1_preprocess import date2student_id
import tqdm
import numpy as np
from scipy.integrate import simps
import math
import os
import json

In [2]:
FREQ_BANDS = {    # 4-7
    "delta": [0.5, 4],   # 1-3 
    "theta": [4, 8],     # 4-7
    "alpha": [8, 13],    # 8-12
    "beta": [13, 30],    # 13-30
    "gamma": [30,50]
}

In [3]:
def bandpower(data, sf, band, method='welch', window_sec=None, relative=False):
    from scipy.signal import welch
    from mne.time_frequency import psd_array_multitaper

    band = np.asarray(band)
    low, high = band

    # Compute the modified periodogram (Welch)
    if method == 'welch':
        if window_sec is not None:
            nperseg = window_sec * sf
        else:
            nperseg = (2 / low) * sf

        freqs, psd = welch(data, sf, nperseg=nperseg)

    elif method == 'multitaper':
        psd, freqs = psd_array_multitaper(data, sf, adaptive=True,
                                          normalization='full', verbose=0)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find index of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using parabola (Simpson's rule)
    bp = simps(psd[idx_band], dx=freq_res)

    if relative:
        bp /= simps(psd, dx=freq_res)
    return bp

In [4]:
'''def get_bp(idx2eeg, out_path):
    print('extracting bp', end = ' ')
    idx2de = {}
    for idx in idx2eeg.keys():
        eeg = np.array(idx2eeg[idx])
        de_list = []
        for i in range(min(int(eeg.shape[1]/1000),10)):
            fs = 1000
            # tmp_data = eeg[:, i * 1000 : i * 1000 + 500]

            tmp_data = eeg[:, i * 1000 : i * 1000 + 1000]
            tmp_fs = []
            for channel_id in range(tmp_data.shape[0]):
                tmp_feature = []
                for band_item in FREQ_BANDS.values():
                    tmp_feature.append(math.log(bandpower(tmp_data[channel_id], fs, band_item)))
                tmp_fs.append(tmp_feature)
            de_list.append(tmp_fs)
        idx2de[idx] = de_list
    json.dump(idx2de, open(out_path, 'w'))
'''

"def get_bp(idx2eeg, out_path):\n    print('extracting bp', end = ' ')\n    idx2de = {}\n    for idx in idx2eeg.keys():\n        eeg = np.array(idx2eeg[idx])\n        de_list = []\n        for i in range(min(int(eeg.shape[1]/1000),10)):\n            fs = 1000\n            # tmp_data = eeg[:, i * 1000 : i * 1000 + 500]\n\n            tmp_data = eeg[:, i * 1000 : i * 1000 + 1000]\n            tmp_fs = []\n            for channel_id in range(tmp_data.shape[0]):\n                tmp_feature = []\n                for band_item in FREQ_BANDS.values():\n                    tmp_feature.append(math.log(bandpower(tmp_data[channel_id], fs, band_item)))\n                tmp_fs.append(tmp_feature)\n            de_list.append(tmp_fs)\n        idx2de[idx] = de_list\n    json.dump(idx2de, open(out_path, 'w'))\n"

In [ ]:
#设置时间窗口、去掉前后的幅度。

In [4]:
def get_bp(idx2eeg, out_path, video_duration=60, fs=1000, start_sec=0, end_sec=30):
    print('Extracting bp...', end=' ')
    idx2de = {}
    try:
        for idx in idx2eeg.keys():
            eeg = np.array(idx2eeg[idx])

            # 计算开始和结束时间对应的样本数
            start_samples = start_sec * fs
            end_samples = end_sec * fs

            # 调整EEG数据，只取视频5秒到25秒的数据
            eeg = eeg[:, start_samples:end_samples]

            de_list = []

            # 计算特征提取的次数，确保不超出调整后的数据长度
            num_of_extraction = min(int(eeg.shape[1] / 1000), end_sec - start_sec)

            for i in range(num_of_extraction):
                # 每次处理1000个样本
                tmp_data = eeg[:, i * 1000: i * 1000 + 1000]
                tmp_fs = []
                for channel_id in range(tmp_data.shape[0]):
                    tmp_feature = []
                    for band_item in FREQ_BANDS.values():
                        # 采用de特征
                        tmp_feature.append(math.log(bandpower(tmp_data[channel_id], fs, band_item,window_sec=1, relative=True)))
                    tmp_fs.append(tmp_feature)
                de_list.append(tmp_fs)
            idx2de[idx] = de_list

        # 使用with语句确保文件正确关闭
        with open(out_path, 'w') as file:
            json.dump(idx2de, file)

    except Exception as e:
        print("An error occurred: ", e)
        # 处理或记录异常，根据需要修改

    print('Done.')


In [5]:
for date in tqdm.tqdm(['LAB1-huqifan','LAB1-cangyueyang','LAB1-hongyurui','LAB1-fanhao','LAB1-dongyimeng','LAB1-houlinzhi','LAB1-jiwenjun','LAB1-lujianing','LAB1-miaoshengze',
             'LAB1-wanfangwei','LAB1-wangxiaoting','LAB1-wangzhengni','LAB1-yangchen','LAB1-zhangxue','LAB1-liangqihang','LAB1-daisiwei',
             'LAB1-zhangyutong','LAB1-mengfanjie','LAB1-zhangchenxi','LAB1-liangyanshu','LAB1-zhaochensong','LAB1-chenxingyu','LAB1-chenrong',
'LAB2-huqifan','LAB2-cangyueyang','LAB2-hongyurui','LAB2-fanhao','LAB2-dongyimeng','LAB2-houlinzhi','LAB2-jiwenjun','LAB2-lujianing','LAB2-miaoshengze',
             'LAB2-wanfangwei','LAB2-wangxiaoting','LAB2-wangzhengni','LAB2-yangchen','LAB2-zhangxue','LAB2-liangqihang','LAB2-daisiwei',
             'LAB2-zhangyutong','LAB2-mengfanjie','LAB2-zhangchenxi','LAB2-liangyanshu','LAB2-zhaochensong','LAB2-chenxingyu','LAB2-chenrong']):
    idx2eeg = json.load(open('./x2eeg/'+date+'_idx2eeg.json'))
    get_bp(idx2eeg, './hope_features/'+date+'_idx2de.json')

  0%|          | 0/46 [00:00<?, ?it/s]

Extracting bp... 

  2%|▏         | 1/46 [02:09<1:36:55, 129.24s/it]

Done.
Extracting bp... 

  4%|▍         | 2/46 [04:31<1:40:30, 137.06s/it]

...

Done.
Extracting bp... 

 98%|█████████▊| 45/46 [1:25:39<01:50, 110.45s/it]

Done.
Extracting bp... 

100%|██████████| 46/46 [1:27:25<00:00, 114.03s/it]

Done.
